<a href="https://colab.research.google.com/github/dntjr41/CV_TermP/blob/main/cv_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 경로 지정


In [1]:
%cd drive/My\ Drive/2022_cv_project
!pwd
!ls -la

/content/drive/My Drive/2022_cv_project
/content/drive/My Drive/2022_cv_project
total 4442010
drwx------ 3 root root       4096 May 24 16:44 checkpoints
drwx------ 6 root root       4096 May 23 10:09 cv_project
-rw------- 1 root root      32676 May 24 16:54 cv_project.ipynb
drwx------ 6 root root       4096 May 23 14:42 cv_project_na
-rw------- 1 root root  454219025 May 23 14:36 cv_project_na.zip
-rw------- 1 root root 4094316129 May 23 09:54 cv_project.zip
drwx------ 3 root root       4096 May 24 10:59 data
drwx------ 2 root root       4096 May 23 13:41 .ipynb_checkpoints
drwx------ 3 root root       4096 May 23 10:09 __MACOSX
drwx------ 6 root root       4096 May 24 15:36 model
drwx------ 6 root root       4096 May 24 16:52 outputs
drwx------ 2 root root       4096 May 24 16:52 __pycache__
drwx------ 3 root root       4096 May 23 14:42 pytorch_ssim
-rw------- 1 root root       1637 May 24 11:06 test.py
-rw------- 1 root root       4716 May 23 09:47 train.py
-rw------- 1 root root   

# 압축 풀기

In [ ]:
import tqdm
import zipfile
import os
import glob

# # 폴더 복사 하기
# import shutil
# shutil.copytree('./cv_project', './cv_project_na')

# # 파일 크기 확인
# filepaths = os.listdir('./cv_project/train')
# print(len(filepaths))
# filepaths = os.listdir('./cv_project/val')
# print(len(filepaths))
# filepaths = os.listdir('./cv_project/mask')
# print(len(filepaths))
# filepaths = os.listdir('./cv_project/hint')
# print(len(filepaths))

# 압축 풀기
file_name = 'cv_project_na'
!unzip -qq '{file_name}'

# 파일 크기 확인
filepaths = os.listdir('./cv_project_na/train')
print(len(filepaths))
filepaths = os.listdir('./cv_project_na/val')
print(len(filepaths))
filepaths = os.listdir('./cv_project_na/mask')
print(len(filepaths))
filepaths = os.listdir('./cv_project_na/hint')
print(len(filepaths))

10000
2000
1000
1000


## 더 빠른 GPU

Colab Pro를 사용하여 가장 빠른 GPU에 우선적으로 액세스하세요. Pro+의 경우 더 빠릅니다. 예를 들어 대부분의 표준 Colab 사용자가 속도가 느린 K80 GPU를 수신할 때 Colab Pro 사용자는 T4 또는 P100 GPU를 이용할 수 있습니다. 언제든지 다음 셀을 실행하여 할당된 GPU를 확인할 수 있습니다.

아래 코드 셀의 실행 결과가 ‘Not connected to a GPU’인 경우 메뉴의 런타임 &gt; 런타임 유형 변경에서 런타임을 변경하여 GPU 가속기를 사용 설정한 다음 코드 셀을 다시 실행하면 됩니다.

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 24 16:52:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 추가 메모리

<p>Colab Pro를 구독하면 고용량 메모리 VM에 액세스할 수 있습니다&#40;사용 가능한 경우&#41;. Pro+에는 더 많은 메모리가 제공됩니다. 고용량 메모리 런타임을 사용하도록 노트북 환경설정을 지정하려면 런타임 &gt; '런타임 유형 변경' 메뉴를 선택한 다음 런타임 구성 드롭다운에서 고용량 RAM을 선택하세요.</p>
<p>언제든지 다음 코드 셀을 실행하여 사용 가능한 메모리 용량을 확인할 수 있습니다.</p>
아래 코드 셀의 실행 결과가 ‘Not using a high-RAM runtime’인 경우 메뉴의 런타임 &gt; 런타임 유형 변경에서 고용량 RAM 런타임을 사용 설정하고 런타임 구성 드롭다운에서 고용량 RAM을 선택한 다음 코드 셀을 다시 실행하면 됩니다.

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# Install pytorch-ssim, mssim
<p> pip install pytorch_msssim </p>
<p> pip install pytorch_ssim </p>

In [1]:
pip install pytorch_msssim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Training

In [ ]:
import os
from data.dataset import ColorHintDataset
import torch.utils.data as data
import torch
import cv2
import tqdm
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from data.transform import tensor2im
from model.res_unet.res_unet import ResUnet
from model.res_unet.res_unet_plus import ResUnetPlusPlus
from model.res_unet.unet import UNet
from model.att_unet.att_unet import Unet
import matplotlib.image as img
import copy, time
from model.sm.model import ResAttdU_Net
from utils import AverageMeter, ssim, save_img

device = "cpu"
if torch.cuda.is_available():
  device = "cuda:0"
  print('device 0 :', torch.cuda.get_device_name(0))


def main():
    # Change to your data root directory
    root_path = "./cv_project"

    check_path = './checkpoints/'
    # Depend on runtime setting
    use_cuda = True

    # make the directory
    os.makedirs('./checkpoints/', exist_ok=True)
    os.makedirs('./outputs/', exist_ok=True)
    os.makedirs('./outputs/test', exist_ok=True)
    os.makedirs('./outputs/GroundTruth', exist_ok=True)
    os.makedirs('./outputs/Hint', exist_ok=True)
    os.makedirs('./outputs/Output', exist_ok=True)
    os.makedirs('./checkpoints', exist_ok=True)

    # Load the data
    train_dataset = ColorHintDataset(root_path, 256, "train")
    val_dataset = ColorHintDataset(root_path, 256, "val")

    dataloaders = {}
    dataloaders['train'] = torch.utils.data.DataLoader(train_dataset, batch_size=4, num_workers=2, shuffle=True)
    dataloaders['valid'] = torch.utils.data.DataLoader(val_dataset, batch_size=4, num_workers=2, shuffle=False)


    print('train dataset: ', len(train_dataset))
    print('validation dataset: ', len(val_dataset))


    # Select the model
    models = {'ResUnet': ResUnet(3), 'ResUnetPlusPlus': ResUnetPlusPlus(3), 'UNet': UNet(), 'ResAttdUnet' : ResAttdU_Net()}
    model = Unet().to(device)

    # load the model
    model.load_state_dict(torch.load('./checkpoints/model-epoch-1-losses-0.02557.pth'))

    criterion = torch.nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    lmbda = lambda epoch : 0.95
    exp_lr_scehduler = optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
    epochs = 30

    # initialize the
    since = time.time()
    train_loss, train_acc, valid_loss, valid_acc = [], [], [], []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 999
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            losses = AverageMeter()

            # Iterate over data.
            for i, data in enumerate(tqdm.tqdm(dataloaders[phase])):
                if use_cuda:
                    l = data["l"].to(device)
                    ab = data["ab"].to(device)
                    hint = data["hint"].to(device)
                else:
                    l = data["l"]
                    ab = data["ab"]
                    hint = data["hint"]

                gt_image = torch.cat((l, ab), dim=1)
                hint_image = torch.cat((l, hint), dim=1)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(hint_image)
                    loss = criterion(outputs, gt_image)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        # zero the parameter gradients
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                losses.update(loss.item(), hint_image.size(0))

                if phase == 'train':
                  if i % 1000 == 0:
                    print('\t Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(loss=losses))
                
                else:
                  out_hint_np = tensor2im(outputs)
                  out_hint_bgr = cv2.cvtColor(out_hint_np, cv2.COLOR_LAB2BGR)

                  hint_np = tensor2im(hint_image)
                  hint_bgr = cv2.cvtColor(hint_np, cv2.COLOR_LAB2BGR)

                  gt_np = tensor2im(gt_image)
                  gt_bgr = cv2.cvtColor(gt_np, cv2.COLOR_LAB2BGR)
                  save_img(gt_bgr, hint_bgr, out_hint_bgr, i)

            if phase == 'train':
                exp_lr_scehduler.step()
                train_loss.append(losses.avg)

            else:
                valid_loss.append(losses.avg)

            print(' {} Loss: {:.3f} '.format(phase, losses.avg))

            # deep copy the model
            if phase == 'valid' and losses.avg < best_loss:
                best_idx = epoch
                best_loss = losses.avg
                best_model_wts = copy.deepcopy(model.state_dict())

                # Save model & checkpoint
                torch.save(model.state_dict(), './checkpoints/model-epoch-{}-losses-{:.5f}.pth'.format(epoch + 1, best_loss))

                print('==> best model saved - %d / %.3f' % (best_idx, best_loss))

    # Training Result
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: %d - %.4f' % (best_idx, best_loss))


    # Plot the training procedure
    epoch_axis = np.arange(0, epochs)
    plt.figure()
    plt.title('LOSS')
    plt.plot(epoch_axis, train_loss, epoch_axis, valid_loss, 'r-')
    plt.legend(['Train', 'Validation'], loc='best')
    plt.show()

if __name__ == '__main__':
    main()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 256, 256]           3,584
       BatchNorm2d-2        [-1, 128, 256, 256]             256
              ReLU-3        [-1, 128, 256, 256]               0
            Conv2d-4        [-1, 128, 256, 256]         147,584
       BatchNorm2d-5        [-1, 128, 256, 256]             256
              ReLU-6        [-1, 128, 256, 256]               0
            Conv2d-7        [-1, 128, 256, 256]           3,584
       BatchNorm2d-8        [-1, 128, 256, 256]             256
       double_conv-9        [-1, 128, 256, 256]               0
        MaxPool2d-10        [-1, 128, 128, 128]               0
           Conv2d-11        [-1, 256, 128, 128]         295,168
      BatchNorm2d-12        [-1, 256, 128, 128]             512
             ReLU-13        [-1, 256, 128, 128]               0
           Conv2d-14        [-1, 256, 1

  0%|          | 1/10000 [00:01<3:08:44,  1.13s/it]

	 Loss 0.0300 (0.0300)	


  2%|▏         | 210/10000 [02:51<2:13:28,  1.22it/s]

# Test

In [ ]:
from data.dataset import ColorHintDataset
import torch
import torch.utils.data as data
import cv2
import tqdm
import os
from data.transform import tensor2im
from model.res_unet.res_unet import ResUnet
from model.res_unet.res_unet_plus import ResUnetPlusPlus
from model.res_unet.unet import UNet

device = "cpu"
if torch.cuda.is_available():
  device = "cuda:0"
  print('device 0 :', torch.cuda.get_device_name(0))
  
def main():
    # Change to your data root directory
    root_path = "./cv_project"
    check_point = './checkpoints/best_model.pt'

    # Depend on runtime setting
    use_cuda = True

    test_dataset = ColorHintDataset(root_path, 256, "test")

    dataloaders = {}
    dataloaders['test'] = torch.utils.data.DataLoader(test_dataset, shuffle=False)
    print('test dataset: ', len(test_dataset))


    # state_dict = torch.load(check_point)
    model = ResUnet(3).to(device)

    model.load_state_dict(torch.load(check_point))

    os.makedirs('outputs/test', exist_ok=True)

    model.eval()
    for i, data in enumerate(tqdm.tqdm(dataloaders['test'])):
        if use_cuda:
            l = data["l"].to(device)
            hint = data["hint"].to(device)
            mask = data["mask"].to(device)
        else:
            l = data["l"]
            hint = data["hint"]
            mask = data["mask"]

        filename = data["file_name"]

        hint_image = torch.cat((l, hint), dim=1)

        output_hint = model(hint_image)
        out_hint_np = tensor2im(output_hint)
        output_bgr = cv2.cvtColor(out_hint_np, cv2.COLOR_LAB2BGR)

        fname = str(filename).replace("['", '')
        fname = fname.replace("']", '')

        cv2.imwrite('./outputs/test/' + str(fname), output_bgr)

if __name__ == '__main__':
    main()

device 0 : Tesla P100-PCIE-16GB
test dataset:  1000


100%|██████████| 1000/1000 [00:38<00:00, 26.25it/s]


# Predict

In [ ]:
from data.dataset import ColorHintDataset
import torch
import torch.utils.data as data
import cv2
import tqdm
import os
from data.transform import tensor2im
from model.res_unet.res_unet import ResUnet
from model.res_unet.res_unet_plus import ResUnetPlusPlus
from model.res_unet.unet import UNet
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import pylab
from test import test

def main():
    ## calculate and save psnr, ssim ##
    with torch.no_grad():
      test()
    # change to your Output data directory
    output_path = "./outputs/Output"
    file_list = os.listdir(output_path)

    ssim = np.zeros(len(file_list))
    psnr = np.zeros(len(file_list))

    for i, img_name in enumerate(file_list):
        # print(img_name)
        name = img_name.replace('.png', '')   # remove '.png'
        temp = name.split('_')
        ssim[i] += float(temp[1].replace('ssim:', ''))
        psnr[i] += float(temp[2].replace('psnr:', ''))

    ssim_avg = sum(ssim)/len(ssim)
    psnr_avg = sum(psnr)/len(psnr)

    print('Average of ssim: {}'.format(ssim_avg))
    print('Average of psnr: {}'.format(psnr_avg))

    np.save(os.path.join('./', 'ssim.npy'), ssim)
    np.save(os.path.join('./', 'psnr.npy'), psnr)

    # plot and save ssim curve
    plt.figure()
    plt.title('ssim')
    pylab.xlim(0, len(file_list) + 1)
    pylab.ylim(0, 1.1)
    plt.plot(range(1, len(file_list) + 1), ssim, label='ssim')
    plt.legend()
    plt.savefig(os.path.join('./', 'ssim.pdf'))
    plt.show()
    plt.close()

    # plot and save psnr curve
    plt.figure()
    plt.title('pnsr')
    pylab.xlim(0, len(file_list) + 1)
    pylab.ylim(0, 100)
    plt.plot(range(1, len(file_list) + 1), psnr, label='psnr')
    plt.legend()
    plt.savefig(os.path.join('./', 'psnr.pdf'))
    plt.show()
    plt.close()

if __name__ == '__main__':
    main()